# Cryptographic hash function Kupyna
## Author: Vadym Tunik

### 1. Cryptographic hash functions Kupyna-256, Kupyna-384, and Kupyna-512.

In [19]:
from Kupyna import Kupyna

for hash_nbits in [256, 384, 512]:
    m = "GLORY_TO_UKRAINE"
    res = Kupyna(hash_nbits).hash(m) 
    print(f"Kupyna-{hash_nbits}: {res}")

Kupyna-256: e605f72cebe8ab45b386a08c8f0a1965a346a1085e7f3a8f64c92e192341d9d8
Kupyna-384: cb577ef639b88bed133dfc02b7333c2bc1725ff01e4bc10aa0643eca00363070917a6e4b67a06dd423fc6caaa5f1afa0
Kupyna-512: b4065ae42a29f59caa6389f53782c29acb577ef639b88bed133dfc02b7333c2bc1725ff01e4bc10aa0643eca00363070917a6e4b67a06dd423fc6caaa5f1afa0


### 2. Avalanche effect.

The avalanche effect: a small change in the input should result in a significant change in the corresponding hash value. This ensures that even similar inputs produce different hash values, making it difficult to establish relationships between inputs based on their hashes.

Pangram: A phrase, expression, or text that contains all letters of the alphabet. 

In [20]:
from numpy.random import randint

pangrams = [
    "Щастям б'єш жук їх глицю в фон й ґедзь пріч",
    "Факт ґринджол: бій псюг вщух, з'їм шче яєць.",
    "З'їв аґрусу — та ягода цілюща б'є жах інфекцій шипучим „ь“.",
    "Фабрикуймо гідність, лящім їжею, ґав хапаймо, з'єднавці чаш!",
    "Юнкерський джинґл, що при безхліб'ї чує фашист, це ловця гімн.",
    "Хвацький юшковар Філіп щодня на ґанку готує сім'ї вечерю з жаб.",
    "В Бахчисараї фельд'єґер зумів одягнути ящірці жовтий капюшон!",
    "На подушечці форми любої є й ґудзик щоб пір'я геть жовте сховати.",
    "Щурячий бугай із їжаком-харцизом в'ючись підписали ґешефт у єнах.",
    "Грішний джиґіт, що хотів у Францію, позбувався цієї думки з'їдаючи трюфель.",
    "Десь чув, що той фраєр привіз їхньому царю грильяж та класну шубу з пір'я ґави.",
    "Жебракують філософи при ґанку церкви в Гадячі, ще й шатро їхнє п'яне знаємо.",
    "Протягом цієї п'ятирічки в ґрунт було висаджено гарбуз, шпинат та цілющий фенхель."
]

def compute_hashes(pangrams, hash_func):
    hashes = []
    for pangram in pangrams:
        hash_value = hash_func.hash(pangram)
        hashes.append(hash_value)
    return hashes

def modify_pangrams(pangrams):
    modified_pangrams = []
    for pangram in pangrams:
        rnd_index = randint(0, len(pangram))
        modified_pangram = pangram[rnd_index+1:] + pangram[:rnd_index]
        modified_pangrams.append(modified_pangram)
    return modified_pangrams

def compare_hashes(original_hashes, modified_hashes):
    compare_results = []
    for orig_hash, mod_hash in zip(original_hashes, modified_hashes):
        compare_results.append(sum(c1 != c2 for c1, c2 in zip(orig_hash, mod_hash)))
    return compare_results


for hash_nbits in [256, 384, 512]:
    hash_func = Kupyna(hash_nbits)

    hashes = compute_hashes(pangrams, hash_func)
    modified_pangrams = modify_pangrams(pangrams)
    new_hashes = compute_hashes(modified_pangrams, hash_func)
    compare_results = compare_hashes(hashes, new_hashes)
    print(f"Kupyna-{hash_nbits}: number of changes = {compare_results}")

Kupyna-256: number of changes = [60, 57, 57, 61, 56, 60, 63, 62, 60, 61, 58, 57, 61]
Kupyna-384: number of changes = [93, 93, 89, 91, 90, 89, 89, 89, 89, 91, 84, 88, 86]
Kupyna-512: number of changes = [118, 123, 124, 120, 125, 119, 119, 117, 121, 120, 119, 119, 121]


### 3. Algorithm for finding partial collisions for the first k bits of the hash sum, 5 ≤ k ≤ 15.

In [22]:
import random
import string
import time

def random_string(length=20):
    """Generate a random string of fixed length."""
    letters = string.ascii_letters + string.digits
    return ''.join(random.choice(letters) for i in range(length))

def find_partial_collision(k, hash_function):
    seen_hashes = {}
    while True:
        s = random_string()
        hashed = hash_function.hash(s)
        prefix = hashed[:k // 4]  # k//4 because 2 hex digits represent 1 byte (8 bits), thus for k bits we need k//4 hex digits
        
        if prefix in seen_hashes and seen_hashes[prefix] != s:
            return (s, seen_hashes[prefix], prefix)
        seen_hashes[prefix] = s

def collision_experiment(hash_function, name, k_values=range(5, 16)):
    results = []
    for k in k_values:
        times = []
        for _ in range(100):
            start_time = time.time()
            collision = find_partial_collision(k, hash_function)
            elapsed_time = time.time() - start_time
            times.append(elapsed_time)
        avg_time = sum(times) / len(times)
        results.append((k, avg_time))
        print(f"{name} {k}-bit: {avg_time:.4f} seconds")
    return results


for hash_nbits in [256, 384, 512]:
    hash_func = Kupyna(hash_nbits)
    results = collision_experiment(hash_func, f"Kupyna-{hash_nbits}")

Kupyna-256 5-bit: 0.0137 seconds
Kupyna-256 6-bit: 0.0140 seconds
Kupyna-256 7-bit: 0.0125 seconds
Kupyna-256 8-bit: 0.0440 seconds
Kupyna-256 9-bit: 0.0457 seconds
Kupyna-256 10-bit: 0.0437 seconds
Kupyna-256 11-bit: 0.0467 seconds
Kupyna-256 12-bit: 0.2059 seconds
Kupyna-256 13-bit: 0.1991 seconds
Kupyna-256 14-bit: 0.1813 seconds
Kupyna-256 15-bit: 0.1922 seconds
Kupyna-384 5-bit: 0.0337 seconds
Kupyna-384 6-bit: 0.0344 seconds
Kupyna-384 7-bit: 0.0330 seconds
Kupyna-384 8-bit: 0.1209 seconds
Kupyna-384 9-bit: 0.1334 seconds
Kupyna-384 10-bit: 0.1331 seconds
Kupyna-384 11-bit: 0.1334 seconds
Kupyna-384 12-bit: 0.4442 seconds
Kupyna-384 13-bit: 0.5302 seconds
Kupyna-384 14-bit: 0.5100 seconds
Kupyna-384 15-bit: 0.4809 seconds
Kupyna-512 5-bit: 0.0356 seconds
Kupyna-512 6-bit: 0.0368 seconds
Kupyna-512 7-bit: 0.0336 seconds
Kupyna-512 8-bit: 0.1260 seconds
Kupyna-512 9-bit: 0.1193 seconds
Kupyna-512 10-bit: 0.1130 seconds
Kupyna-512 11-bit: 0.1310 seconds
Kupyna-512 12-bit: 0.4997 sec